I utilized the LSTM implementation specifically tailored for EEG data from the following GitHub repository: https://github.com/theyou21/BigProject. This resource provided invaluable support for my LSTM analysis.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
ec_data_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC"
eo_data_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EO"
ec_eeg_data = np.load(os.path.join(ec_data_dir, "normalized_epoch_eeg_data.npy"))
eo_eeg_data = np.load(os.path.join(eo_data_dir, "normalized_epoch_eeg_data.npy"))

In [ ]:
print(ec_eeg_data.shape)
print(eo_eeg_data.shape)

(4356, 1, 32, 4975)
(4344, 1, 32, 4975)


In [ ]:
ec_labels_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC"
eo_labels_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EO"
ec_eeg_labels = np.load(os.path.join(ec_labels_dir, "labels_data.npy"))
eo_eeg_labels = np.load(os.path.join(eo_labels_dir, "labels_data.npy"))

In [ ]:
print(ec_eeg_labels.shape)
print(eo_eeg_labels.shape)

(4356, 2)
(4344, 2)


In [ ]:
for label in ec_eeg_labels:
  sample_id = label[0]
  if sample_id not in eo_eeg_labels[:, 0]:
        index_to_remove = np.where(ec_eeg_labels[:, 0] == sample_id)[0]
        ec_eeg_labels = np.delete(ec_eeg_labels, index_to_remove, axis=0)
        ec_eeg_data = np.delete(ec_eeg_data, index_to_remove, axis=0)
print(ec_eeg_labels.shape)
print(ec_eeg_data.shape)

eeg_data = np.concatenate((ec_eeg_data[:, 0], eo_eeg_data[:, 0]), axis=1)
eeg_data.shape

(4344, 2)
(4344, 1, 32, 4975)


(4344, 64, 4975)

In [ ]:
eeg_labels = ec_eeg_labels

In [ ]:
healthy_count, mdd_count = 0, 0
for sample in eeg_labels:
  if sample[1] == "MDD":
      mdd_count += 1
  else:
      healthy_count += 1

print(f"Number of MDD participants: {mdd_count}")
print(f"Number of Healthy participants: {healthy_count}")

Number of MDD participants: 3780
Number of Healthy participants: 564


Extracting data for male participants

In [ ]:
df_participants = pd.read_pickle('/content/drive/MyDrive/TD-BRAIN/TDBRAIN_participants_V2_data/df_participants.pkl')
male_count = 0

eeg_data_male = []
eeg_label_male = []

for i, labels in enumerate(eeg_labels):
     sample_id = labels[0]
     index = df_participants.loc[df_participants['participants_ID'] == sample_id].index
     if [value for value in df_participants.loc[index, 'gender']][0] == 0:
       eeg_data_male.append(eeg_data[i])
       eeg_label_male.append(eeg_labels[i])

eeg_data_male = np.array(eeg_data_male)
eeg_label_male = np.array(eeg_label_male)

print(eeg_data_male.shape, eeg_label_male.shape)

(2412, 64, 4975) (2412, 2)


In [ ]:
healthy_count_male, mdd_count_male = 0, 0
for sample in eeg_label_male:
  if sample[1] == "MDD":
      mdd_count_male += 1
  else:
      healthy_count_male += 1

print(f"Number of MDD male participants: {mdd_count_male}")
print(f"Number of Healthy male participants: {healthy_count_male}")

Number of MDD male participants: 2040
Number of Healthy male participants: 372


### **Converting the labels to binary**
1 -> MDD

0 -> HEALTHY

# **Model**

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.metrics import Precision, Recall
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
ll = eeg_label_male
encountered_sample_ids = {}
sample_ids_with_more_than_12_entries = []

for index, sample_id in enumerate(ll):
    sample_id_tuple = tuple(sample_id)
    count = encountered_sample_ids.get(sample_id_tuple, 0)
    count += 1
    encountered_sample_ids[sample_id_tuple] = count
    if count > 12:
        sample_ids_with_more_than_12_entries.append((sample_id_tuple, index))

indices_to_remove = [index for _, index in sample_ids_with_more_than_12_entries]
eeg_label_male = [sample for i, sample in enumerate(eeg_label_male) if i not in indices_to_remove]
eeg_data_male = [data for i, data in enumerate(eeg_data_male) if i not in indices_to_remove]
print("Length of filtered eeg_label_male:", len(eeg_label_male))
print("Length of filtered eeg_data_male:", len(eeg_data_male))


###### Undersampling and preparing training data ########
ll = eeg_label_male
unique_sample_id = []
encountered_sample_ids = set()
print(len(ll))
for sample_id in ll:
    sample_id_tuple = tuple(sample_id)
    if sample_id_tuple not in encountered_sample_ids:
        unique_sample_id.append(sample_id)
        encountered_sample_ids.add(sample_id_tuple)
print(len(unique_sample_id))

num_samples_minority = 31
indices_maj = [index for index, sample in enumerate(unique_sample_id) if sample[1] == "MDD"]
indices_min = [index for index, sample in enumerate(unique_sample_id) if sample[1] == "HEALTHY"]
undersampled = np.random.choice(indices_maj, num_samples_minority, replace=False)

balanced_data_indices = np.concatenate([indices_min, undersampled])
balanced_unique_sample_id = [unique_sample_id[i] for i in balanced_data_indices]

# Extract all unique sample IDs from balanced_unique_sample_id
unique_sample_ids = [sample_id[0] for sample_id in balanced_unique_sample_id]
print(len(unique_sample_ids))
# Extract all indices from eeg_label_male for sample IDs in balanced_unique_sample_id
indices = []
for i, sample_id in enumerate(eeg_label_male):
  if sample_id[0] in unique_sample_ids:
        indices.append(i)

# Convert indices to a NumPy array
indices = np.array(indices)
X_train = []
y_train = []
for i in indices:
    X_train.append(eeg_data_male[i])
    y_train.append(eeg_label_male[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

# Shuffle together with their indices
permutation = np.random.permutation(len(X_train))
X_train = X_train[permutation]
y_train = y_train[permutation]

print(X_train.shape)
# print(y_train)

sample_ids = []
for sample in y_train:
  sample_ids.append(sample[0])
sample_ids = np.array(sample_ids)
l = np.array([1 if label[1] == "MDD" else 0 for label in y_train])


Length of filtered eeg_label_male: 2328
Length of filtered eeg_data_male: 2328
2328
194
62
(744, 64, 4975)


In [ ]:
from sklearn.metrics import f1_score, confusion_matrix
from keras.callbacks import ModelCheckpoint

class EEGClassifier:
    def __init__(self, input_shape=(64, 4975), lstm_units=128):
        self.input_shape = input_shape
        self.lstm_units = lstm_units
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(LSTM(units=self.lstm_units,
                   input_shape=self.input_shape,
                   return_sequences=True))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(LSTM(units=64,
                   return_sequences=True))
        model.add(Dropout(0.2))

        model.add(LSTM(units=32))
        model.add(Dropout(0.2))

        model.add(Dense(units=64, activation='sigmoid'))
        model.add(Dropout(0.2))

        model.add(Dense(units=1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall()])
        return model

    def train(self, X_train, y_train, X_val, y_val, epochs=5, batch_size=1):
        # Calculate class weights
        class_weights = {0: 1, 1: 1}  # Initialize with equal weights
        num_minority = np.sum(y_train == 0)
        num_majority = np.sum(y_train == 1)
        total_samples = len(y_train)
        class_weights[0] = (1 / num_minority) * (total_samples / 2.0)
        class_weights[1] = (1 / num_majority) * (total_samples / 2.0)

        history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), class_weight=class_weights, verbose=0)
        return history

    def evaluate(self, X_test, y_test):
        loss, accuracy, precision, recall = self.model.evaluate(X_test, y_test, verbose=0)
        print(f'Test Loss: {loss}, Test Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}')
        y_pred = self.model.predict(X_test)
        y_pred_classes = np.round(y_pred)
        f1 = f1_score(y_test, y_pred_classes)
        print(f'F1 Score: {f1}')
        cm = confusion_matrix(y_test, y_pred_classes)
        print('Confusion Matrix:')
        print(cm)

        evaluation_metrics = {
            'loss': loss,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'confusion_matrix': cm
        }

        return evaluation_metrics

    def predict(self, X):
        return self.model.predict(X)

    def plot_loss(self, history):
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()


In [ ]:
from sklearn.model_selection import StratifiedKFold

def main():
    global classifier
    classifier = EEGClassifier()

    num_splits = 5
    cv = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    for fold_idx, (train_index, val_index) in enumerate(cv.split(X_train, l), 1):
        print(f"Fold {fold_idx}:")

        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = l[train_index], l[val_index]

        history = classifier.train(X_train_fold, y_train_fold, X_val_fold, y_val_fold)

        # Evaluate on training set after training
        train_metrics = classifier.evaluate(X_train_fold, y_train_fold)
        print(f'Training Results - Loss: {train_metrics["loss"]}, Accuracy: {train_metrics["accuracy"]}, '
              f'Precision: {train_metrics["precision"]}, Recall: {train_metrics["recall"]}, '
              f'F1 Score: {train_metrics["f1_score"]}')

        # Evaluate on the validation set after training
        val_metrics = classifier.evaluate(X_val_fold, y_val_fold)
        print(f'Validation Results - Loss: {val_metrics["loss"]}, Accuracy: {val_metrics["accuracy"]}, '
              f'Precision: {val_metrics["precision"]}, Recall: {val_metrics["recall"]}, '
              f'F1 Score: {val_metrics["f1_score"]}')
        print()

if __name__ == "__main__":
    main()

Fold 1:
Test Loss: 0.5082722902297974, Test Accuracy: 0.7983193397521973, Precision: 0.7673715949058533, Recall: 0.8552188277244568
19/19 [==============================] - 1s 14ms/step
F1 Score: 0.8089171974522293
Confusion Matrix:
[[221  77]
 [ 43 254]]
Training Results - Loss: 0.5082722902297974, Accuracy: 0.7983193397521973, Precision: 0.7673715949058533, Recall: 0.8552188277244568, F1 Score: 0.8089171974522293
Test Loss: 1.3394775390625, Test Accuracy: 0.4899328947067261, Precision: 0.49450549483299255, Recall: 0.6000000238418579
5/5 [==============================] - 0s 15ms/step
F1 Score: 0.5421686746987951
Confusion Matrix:
[[28 46]
 [30 45]]
Validation Results - Loss: 1.3394775390625, Accuracy: 0.4899328947067261, Precision: 0.49450549483299255, Recall: 0.6000000238418579, F1 Score: 0.5421686746987951

Fold 2:
Test Loss: 0.48095595836639404, Test Accuracy: 0.8453781604766846, Precision: 0.8451178669929504, Recall: 0.8451178669929504
19/19 [==============================] - 0s 